In [1]:
""" Convert RDS table to csv files on S3 and GCS
-------------------------------------------------------------------------------

update 2020/02/10 update output 11-12 input 8-9

Author: Rutger Hofste
Date: 20180712
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04
"""

# imports
import re
import os
import numpy as np
import pandas as pd
from retrying import retry
from datetime import timedelta
from sqlalchemy import *
pd.set_option('display.max_columns', 500)
import multiprocessing

SCRIPT_NAME = 'Y2018M07D17_RH_RDS_To_S3_V02'
OUTPUT_VERSION = 12

TESTING = 0

DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"

INPUT_TABLE_NAME = "y2018m07d30_rh_coalesce_columns_v01_v09"

ec2_output_path = "/volumes/data/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
gcs_output_path = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input Table: " , INPUT_TABLE_NAME,
      "\nOutput ec2: ", ec2_output_path,
      "\nOutput s3: " , s3_output_path,
      "\nOutput gcs: ", gcs_output_path)

Input Table:  y2018m07d30_rh_coalesce_columns_v01_v09 
Output ec2:  /volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12/ 
Output s3:  s3://wri-projects/Aqueduct30/processData/Y2018M07D17_RH_RDS_To_S3_V02/output_V12/ 
Output gcs:  gs://aqueduct30_v01/Y2018M07D17_RH_RDS_To_S3_V02/output_V12/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2020M02D10 UTC 17:50


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12/': No such file or directory


In [4]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,DATABASE_ENDPOINT,DATABASE_NAME))

In [5]:
cpu_count = multiprocessing.cpu_count()
print("Power to the maxxx:", cpu_count)

Power to the maxxx: 16


In [6]:
sql = "SELECT DISTINCT pfafid_30spfaf06 FROM {} ORDER BY pfafid_30spfaf06".format(INPUT_TABLE_NAME)

In [7]:
df = pd.read_sql(sql,engine)

In [8]:
df.shape

(16385, 1)

In [9]:
df.head()

,pfafid_30spfaf06
0,111011
1,111012
2,111013
3,111014
4,111015


In [10]:
if TESTING:
    df = df[0:10]

In [11]:
df_split = np.array_split(df, cpu_count*100)

In [12]:
def basin_to_csv(df):
    for index, row in df.iterrows():
        pfafid = row["pfafid_30spfaf06"]
        sql = "SELECT * FROM {} WHERE pfafid_30spfaf06 = {}".format(INPUT_TABLE_NAME,pfafid)
        df_basin = pd.read_sql(sql,engine)
        now = datetime.datetime.now()
        df_basin["processed_timestamp"] = pd.Timestamp(now)  
        output_file_name = "{}_V{:02.0f}.csv".format(pfafid,OUTPUT_VERSION)
        output_file_path = "{}/{}".format(ec2_output_path,output_file_name)
        # Added on Y2018M07D30, convert to numeric type if possible
        df_basin2 = df_basin.apply(pd.to_numeric,errors='ignore')
        df_basin2.to_csv(output_file_path)
        print(output_file_path)

In [13]:
# cleared output to save space
p= multiprocessing.Pool()
results_buffered = p.map(basin_to_csv,df_split)
p.close()
p.join()

OperationalError: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
 [SQL: 'SELECT * FROM y2018m07d30_rh_coalesce_columns_v01_v09 WHERE pfafid_30spfaf06 = 115801'] (Background on this error at: http://sqlalche.me/e/e3q8)

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//155562_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111011_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122784_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//243560_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//232144_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//223940_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132895_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//153021_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//131902_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//145102_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//181976_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//157807_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//127443_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//162203_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111019_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//224002_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//232160_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//243802_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//131909_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//162300_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132930_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//145109_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//155572_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//142633_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//158116_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//127450_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122820_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//172524_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//127467_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//158143_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//232240_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//183000_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//145207_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//155580_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132947_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//172541_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132170_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122846_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111046_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//153036_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//142642_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//162444_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132955_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//127484_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//142649_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//232302_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//211015_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//155618_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//145404_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132224_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//172548_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111081_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//153044_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//244206_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122854_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//127485_V12.csv
/volumes/data/Y2018M

...

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//662905_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//652407_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//772341_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//723060_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//726013_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//822476_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//742984_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//863099_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//812105_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//742441_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//753302_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//831956_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//782695_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//662906_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//772349_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//723078_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//812204_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//742449_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//664023_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//911213_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//753401_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//713221_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//822494_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//742993_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//782704_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//726040_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//832101_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//772350_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//713229_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//782803_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//726076_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//812304_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//743003_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//772404_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//911263_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//664043_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//723116_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//742486_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//822514_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//753501_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//832200_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//743004_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//726086_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//782920_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//751203_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//812440_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//832427_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//753610_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//772603_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//911287_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//742495_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//822542_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//726087_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//782930_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//751204_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//911288_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//753640_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//822590_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//772805_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//782997_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//812490_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//832460_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//753650_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//751406_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//822610_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//772806_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//812501_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//782998_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//832470_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//753660_V12.csv
/volumes/data/Y2018M

In [14]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive --quiet

In [15]:
# cleared output to save space
!gsutil -m cp \
{ec2_output_path}/*.csv \
{gcs_output_path}

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111011_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111012_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111014_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111013_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111015_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111016_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111018_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111017_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111019_V12.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//112130_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//112140_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//112150_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//112120_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//112180_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//112200_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//112190_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//112400_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//112300_V12.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//114442_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//114443_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//114444_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//114445_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//114446_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//114447_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//114448_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//114449_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//114450_V12.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//115426_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//115427_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//115428_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//115429_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//115430_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//115440_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//115451_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//115452_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//115453_V12.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122859_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122860_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122870_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122880_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122890_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122911_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122912_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122913_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122914_V12.csv [Content-Type=text/

...

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//863001_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//863002_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//863003_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//863004_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//863005_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//863006_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//863007_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//863008_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//863009_V12.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//911405_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//911406_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//911407_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//911408_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//911409_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//911500_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//911510_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//911520_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//911530_V12.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//912300_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//912400_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//912510_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//912520_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//912530_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//912540_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//912551_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//912552_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//912553_V12.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//913478_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//913479_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//913480_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//913490_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//913500_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//913510_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//913520_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//913530_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//913540_V12.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//914703_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//914704_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//914705_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//914706_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//914707_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//914708_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//914801_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//914709_V12.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//914802_V12.csv [Content-Type=text/

In [16]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

1:03:56.797800


Previous runs:  
0:42:01.416597  
0:42:14.565295  
0:31:40.728932  
0:23:35.132388

